In [ ]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
import json

In [ ]:
# !pip install -q --upgrade git+https://github.com/SafeGraphInc/safegraph_py

# from safegraph_py_functions import safegraph_py_functions as sgpy

# sgpy.test_me_sgpy() # returns 'Hello World' to ensure you have downloaded the library
# sgpy.help_sgpy() # returns a list of all active functions and their arguments in the safegraph_py library

### Mount Google Drive

In [ ]:
from google.colab import drive as mountGoogleDrive 
mountGoogleDrive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Explore JSON file structure

In [ ]:
example_path = r'/content/drive/MyDrive/Research/Safegraph/data/2018-01-01through2018-01-08/New York,NYfrom2018-01-01to2018-01-08PART1.json'
f = open(example_path)
a = json.load(f)

In [ ]:
example_df = pd.json_normalize(a['data']['search']['places']['results']['edges'])

In [ ]:
example_df['node.weekly_patterns'].str[0].str['visitor_home_cbgs']

0      None
1      None
2      None
3      None
4      None
       ... 
495    None
496    None
497    None
498      {}
499      {}
Name: node.weekly_patterns, Length: 500, dtype: object

In [ ]:
example_df.columns

Index(['node.placekey', 'node.safegraph_core.placekey',
       'node.safegraph_core.parent_placekey',
       'node.safegraph_core.location_name', 'node.safegraph_core.brands',
       'node.safegraph_core.top_category', 'node.safegraph_core.sub_category',
       'node.safegraph_core.naics_code', 'node.safegraph_core.latitude',
       'node.safegraph_core.longitude', 'node.safegraph_core.street_address',
       'node.safegraph_core.city', 'node.safegraph_core.region',
       'node.safegraph_core.postal_code',
       'node.safegraph_core.iso_country_code',
       'node.safegraph_core.phone_number', 'node.safegraph_core.open_hours',
       'node.safegraph_core.category_tags', 'node.safegraph_core.opened_on',
       'node.safegraph_core.closed_on',
       'node.safegraph_core.tracking_closed_since',
       'node.safegraph_core.geometry_type', 'node.weekly_patterns',
       'node.safegraph_geometry.polygon_wkt',
       'node.safegraph_geometry.polygon_class',
       'node.safegraph_geometry.

In [ ]:
example_df[example_df['node.placekey'] == 'zzw-226@627-s6m-rtv']

,node.placekey,node.safegraph_core.placekey,node.safegraph_core.parent_placekey,node.safegraph_core.location_name,node.safegraph_core.brands,node.safegraph_core.top_category,node.safegraph_core.sub_category,node.safegraph_core.naics_code,node.safegraph_core.latitude,node.safegraph_core.longitude,...,node.safegraph_core.closed_on,node.safegraph_core.tracking_closed_since,node.safegraph_core.geometry_type,node.weekly_patterns,node.safegraph_geometry.polygon_wkt,node.safegraph_geometry.polygon_class,node.safegraph_geometry.includes_parking_lot,node.safegraph_geometry.is_synthetic,node.safegraph_geometry.enclosed,node.safegraph_geometry.wkt_area_sq_meters


In [ ]:
example_df.columns

Index(['node.placekey', 'node.safegraph_core.placekey',
       'node.safegraph_core.parent_placekey',
       'node.safegraph_core.location_name', 'node.safegraph_core.brands',
       'node.safegraph_core.top_category', 'node.safegraph_core.sub_category',
       'node.safegraph_core.naics_code', 'node.safegraph_core.latitude',
       'node.safegraph_core.longitude', 'node.safegraph_core.street_address',
       'node.safegraph_core.city', 'node.safegraph_core.region',
       'node.safegraph_core.postal_code',
       'node.safegraph_core.iso_country_code',
       'node.safegraph_core.phone_number', 'node.safegraph_core.open_hours',
       'node.safegraph_core.category_tags', 'node.safegraph_core.opened_on',
       'node.safegraph_core.closed_on',
       'node.safegraph_core.tracking_closed_since',
       'node.safegraph_core.geometry_type', 'node.weekly_patterns',
       'node.safegraph_geometry.polygon_wkt',
       'node.safegraph_geometry.polygon_class',
       'node.safegraph_geometry.

In [ ]:
example_naics_weekly_patterns = example_df[['node.safegraph_core.naics_code','node.weekly_patterns','node.safegraph_geometry.wkt_area_sq_meters']]
example_naics_weekly_patterns.head()

,node.safegraph_core.naics_code,node.weekly_patterns,node.safegraph_geometry.wkt_area_sq_meters
0,722515.0,None,2110.0
1,485113.0,None,NaN
2,813110.0,None,10819.0
3,5412.0,None,11473.0
4,485113.0,None,NaN


In [ ]:
for i in range(7):
  visits = example_naics_weekly_patterns['node.weekly_patterns'].str[0].str['visits_by_day'].str[i].str['visits']
  example_naics_weekly_patterns[f'day_{i}_visits'] = visits

<ipython-input-47-0ca7dab354cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  example_naics_weekly_patterns[f'day_{i}_visits'] = visits


In [ ]:
example_naics_weekly_patterns.fillna(0, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [ ]:
example_naics_weekly_patterns.head()

,node.safegraph_core.naics_code,node.weekly_patterns,node.safegraph_geometry.wkt_area_sq_meters,day_0_visits,day_1_visits,day_2_visits,day_3_visits,day_4_visits,day_5_visits,day_6_visits
0,722515.0,0,2110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,485113.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,813110.0,0,10819.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5412.0,0,11473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,485113.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
week_dirs = os.listdir("/content/drive/MyDrive/Research/Safegraph/NYC/")

In [ ]:
weekly_df = pd.DataFrame()
# for dir in week_dirs:
dir = '2018-01-01through2018-01-08'
files = os.listdir(f"/content/drive/MyDrive/Research/Safegraph/NYC/{dir}")
for file in files:
  f = open(f"drive/MyDrive/Research/Safegraph/NYC/{dir}/{file}")
  a = json.load(f)
  df = pd.json_normalize(a['data']['search']['places']['results']['edges'])
  sub_df = df[['node.safegraph_core.naics_code','node.weekly_patterns']]
  for i in range(7):
    visits = sub_df['node.weekly_patterns'].str[0].str['visits_by_day'].str[i].str['visits']
    sub_df[f'day_{i}_visits'] = visits
  sub_df.fillna(0, inplace=True)
  sub_df.drop('node.weekly_patterns',axis=1, inplace=True)
  weekly_df = pd.concat([weekly_df, sub_df])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors

In [ ]:
weekly_df.head()

,node.safegraph_core.naics_code,day_0_visits,day_1_visits,day_2_visits,day_3_visits,day_4_visits,day_5_visits,day_6_visits
0,722515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,485113.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,813110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5412.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,485113.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
weekly_df.shape[0]

83500

In [ ]:
weekly_df['week_total_visits'] = weekly_df['day_0_visits'] + weekly_df['day_1_visits'] + weekly_df['day_2_visits'] + weekly_df['day_3_visits'] + weekly_df['day_4_visits'] + weekly_df['day_5_visits'] + weekly_df['day_6_visits']

In [ ]:
len(weekly_df.loc[weekly_df['week_total_visits'] == 0])

58368

In [ ]:
f"{np.round(len(weekly_df.loc[weekly_df['week_total_visits'] == 0]) / weekly_df.shape[0] *100,2)}% of POIS had 0 visits"

'69.9% of POIS had 0 visits'

In [ ]:
for i in range(7):
  print(np.round(sum(weekly_df[f'day_{i}_visits']) / sum(weekly_df['week_total_visits'])*100,2))

11.75
9.81
21.36
12.04
18.51
14.44
12.08


In [ ]:
len(weekly_df['node.safegraph_core.naics_code'].unique())

310

In [ ]:
weekly_df['node.safegraph_core.naics_code'].unique().astype(int)

array([722515, 485113, 813110,   5412,   4481, 446120, 621111, 442110,
        81211, 812199, 812930, 611310, 531110, 712110, 722410, 722511,
         5416, 517312, 522110, 512240, 621340, 621330, 523930, 446110,
       812191, 811430, 922110, 443142, 722513, 812910, 492110, 621210,
       531311, 448310, 448140, 611699, 311811, 517919, 812320,   5418,
       611110,   3231, 445110, 621320, 531190, 611691, 445292, 448320,
         3399, 448210,   5151, 451120, 721110, 453920,   6215, 453991,
       712130, 562211,      0,   2382, 451130, 621399, 922120, 453310,
       448120, 621310, 541940, 712190,  54192, 713940, 446191, 531130,
       713210, 485112, 453910, 423730, 622110, 922140, 622310, 811490,
       531210,    487, 611630, 811211, 713990, 445120, 445310, 424210,
       441120,   4422, 447190, 522390, 624410, 339950, 451211, 811412,
         5616, 812112,   6214, 524210, 812220, 453110, 488410, 442299,
       519120, 444190, 532289,   8111, 522320,   6115, 531120, 453210,
      

In [ ]:
weekly_df.loc[weekly_df['week_total_visits'] == weekly_df['week_total_visits'].max()]

,node.safegraph_core.naics_code,day_0_visits,day_1_visits,day_2_visits,day_3_visits,day_4_visits,day_5_visits,day_6_visits,week_total_visits
74,488119.0,2527.0,1630.0,3059.0,558.0,2419.0,2358.0,2938.0,15489.0


In [ ]:
weekly_df.groupby(by='node.safegraph_core.naics_code')

In [ ]:
unique_naics_sum_df = weekly_df.groupby('node.safegraph_core.naics_code').sum()

# ETL Pipeline
Load JSON in batches and transform to data frame with relevant visits, visitors, dwell time, and square foot features.

In [ ]:
root = r'/content/drive/MyDrive/Research/Safegraph/data/'
# root = r'/c/Users/hyone/Google Drive (nhw2114@columbia.edu)/Research/Safegraph/'
out_dir = r'/content/drive/MyDrive/Research/Safegraph/scratch/20221205/'

week_dirs = os.listdir(root)

print(f"Total number of directories {len(week_dirs):,}, one for each week") 

Total number of directories 224, one for each week


In [ ]:
week_dirs.sort()

In [ ]:
count = 0

for dirpath, dirnames, filenames in os.walk(root):
    for file in filenames:
        count += 1

print(f"Total number of files {count:,}") 

Total number of files 37,651


In [ ]:
batchsize = 25
batch_list = []
for i in range(0, len(week_dirs), batchsize):
    batch = week_dirs[i:i+batchsize]
    batch_list.append(batch)

In [ ]:
from datetime import datetime

def get_zipcode(dirs):

  bad_files = []
  weekly_df = pd.DataFrame()

  for dir in tqdm.tqdm(dirs, desc="directory", position=0):
    start, end = str.split(dir, 'through')
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')
    files = os.listdir(f"{root}/{dir}")
    for file in files:
      f = open(f"{root}/{dir}/{file}")
      try:
        a = json.load(f)
        df = pd.json_normalize(a['data']['search']['places']['results']['edges'])
      except:
        print(file)
        bad_files.append(file)
        continue
      
      def getValues(arr):
        if arr is not None:
          return list(arr.values())
        else:
          return [np.nan for i in range(7)]
      
      # get zipcode, cbg and visitor home cbg
      sub_df = df.loc[:,['node.placekey','node.safegraph_core.postal_code','node.weekly_patterns']]
      sub_df['poi_cbg'] = sub_df['node.weekly_patterns'].str[0].str['poi_cbg']
      sub_df['visitor_home_cbgs'] = sub_df['node.weekly_patterns'].str[0].str['visitor_home_cbgs']
      sub_df['visitor_home_aggregation'] = sub_df['node.weekly_patterns'].str[0].str['visitor_home_aggregation']

      sub_df.drop(columns=['node.weekly_patterns'], inplace=True)
      sub_df['start'] = start
      sub_df['end'] = end
      weekly_df = pd.concat([weekly_df, sub_df])
  
  filename_pre = f"zipcode_{dirs[0]}_{dirs[-1]}"
  path = f"{out_dir}/visitor_home_aggregation/{filename_pre}.csv"
  weekly_df.to_csv(path)

  with open(f'{out_dir}/zipcode_{filename_pre}_bad_files.pickle', 'wb') as handle:
    pickle.dump(bad_files, handle, protocol=pickle.HIGHEST_PROTOCOL)

  print(f"DONE with batch ending with {dirs[-1]}")

  return weekly_df


def transform_dataframe(dirs):

  bad_files = []
  weekly_df = pd.DataFrame()

  for dir in tqdm.tqdm(dirs, desc="directory", position=0):
    start, end = str.split(dir, 'through')
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')
    files = os.listdir(f"{root}/{dir}")
    for file in files:
      f = open(f"{root}/{dir}/{file}")
      try:
        a = json.load(f)
        df = pd.json_normalize(a['data']['search']['places']['results']['edges'])
      except:
        print(file)
        bad_files.append(file)
        continue
      
      def getValues(arr):
        if arr is not None:
          return list(arr.values())
        else:
          return [np.nan for i in range(7)]
      
      # get visits by day of week
      sub_df = df.loc[:,['node.placekey','node.safegraph_core.naics_code','node.safegraph_core.location_name','node.weekly_patterns', 'node.safegraph_core.top_category', 'node.safegraph_core.sub_category', 'node.safegraph_geometry.wkt_area_sq_meters']]
      for i in range(7):
        visits = sub_df['node.weekly_patterns'].str[0].str['visits_by_day'].str[i].str['visits']
        sub_df.loc[:,f'day_{i}_visits'] = visits
      raw_visitor_counts = sub_df['node.weekly_patterns'].str[0].str['raw_visitor_counts']
      raw_visit_counts = sub_df['node.weekly_patterns'].str[0].str['raw_visit_counts']
      sub_df.loc[:,'raw_visitor_counts'] = raw_visitor_counts
      sub_df.loc[:,'raw_visit_counts'] = raw_visit_counts

      # get median dwell
      median_dwell = sub_df['node.weekly_patterns'].str[0].str['median_dwell']
      sub_df['median_dwell'] = median_dwell
      buckets = ['<5', '5-10', '11-20', '21-60', '61-120', '121-240', '>240']
      dwell_bucket_values = sub_df['node.weekly_patterns'].str[0].str['bucketed_dwell_times'].apply(getValues)
      v = dwell_bucket_values.values.tolist()
      sub_df.loc[:,buckets] = pd.DataFrame(v, sub_df.index, buckets)

      sub_df.drop(columns=['node.weekly_patterns'], inplace=True)
      sub_df['start'] = start
      sub_df['end'] = end
      weekly_df = pd.concat([weekly_df, sub_df])
  
  filename_pre = f"{dirs[0]}_{dirs[-1]}"
  path = f"{out_dir}/{filename_pre}.csv"
  weekly_df.to_csv(path)

  with open(f'{out_dir}/{filename_pre}_bad_files.pickle', 'wb') as handle:
    pickle.dump(bad_files, handle, protocol=pickle.HIGHEST_PROTOCOL)

  print(f"DONE with batch ending with {dirs[-1]}")

  return weekly_df

In [ ]:
[b[0] + b[-1] for b in batch_list]

['2018-01-01through2018-01-082018-06-25through2018-07-02',
 '2018-07-02through2018-07-092018-12-17through2018-12-24',
 '2018-12-24through2018-12-312019-06-10through2019-06-17',
 '2019-06-17through2019-06-242019-12-02through2019-12-09',
 '2019-12-09through2019-12-162020-05-25through2020-06-01',
 '2020-06-01through2020-06-082020-11-16through2020-11-23',
 '2020-11-23through2020-11-302021-05-10through2021-05-17',
 '2021-05-17through2021-05-242021-11-01through2021-11-08',
 '2021-11-08through2021-11-152022-04-18through2022-04-25']

## Get Zipcode and CBG

In [ ]:
a = get_zipcode(batch_list[0])

directory: 100%|██████████| 25/25 [24:51<00:00, 59.66s/it]


DONE with batch ending with 2018-06-25through2018-07-02


In [ ]:
a = get_zipcode(batch_list[1])

directory: 100%|██████████| 25/25 [25:44<00:00, 61.77s/it]


DONE with batch ending with 2018-12-17through2018-12-24


In [ ]:
a = get_zipcode(batch_list[2])

directory: 100%|██████████| 25/25 [27:01<00:00, 64.86s/it]


DONE with batch ending with 2019-06-10through2019-06-17


In [ ]:
a = get_zipcode(batch_list[3])

directory: 100%|██████████| 25/25 [27:17<00:00, 65.50s/it]


DONE with batch ending with 2019-12-02through2019-12-09


In [ ]:
a = get_zipcode(batch_list[4])

directory: 100%|██████████| 25/25 [25:50<00:00, 62.01s/it]


DONE with batch ending with 2020-05-25through2020-06-01


In [ ]:
a = get_zipcode(batch_list[5])

directory: 100%|██████████| 25/25 [24:18<00:00, 58.35s/it]


DONE with batch ending with 2020-11-16through2020-11-23


In [ ]:
a = get_zipcode(batch_list[6])

directory: 100%|██████████| 25/25 [23:56<00:00, 57.47s/it]


DONE with batch ending with 2021-05-10through2021-05-17


In [ ]:
a = get_zipcode(batch_list[7])

directory:  20%|██        | 5/25 [02:44<11:25, 34.25s/it]

New York,NYfrom2021-06-14to2021-06-21PART51.json


directory:  32%|███▏      | 8/25 [04:49<11:19, 39.95s/it]

New York,NYfrom2021-07-05to2021-07-12PART138.json
New York,NYfrom2021-07-12to2021-07-19PART7.json


directory:  96%|█████████▌| 24/25 [22:49<01:20, 80.80s/it]

New York,NYfrom2021-11-01to2021-11-08PART49.json


directory: 100%|██████████| 25/25 [24:17<00:00, 58.30s/it]


DONE with batch ending with 2021-11-01through2021-11-08


In [ ]:
a = get_zipcode(batch_list[8])

directory:   4%|▍         | 1/24 [00:31<11:59, 31.28s/it]

New York,NYfrom2021-11-08to2021-11-15PART70.json


directory: 100%|██████████| 24/24 [26:18<00:00, 65.77s/it] 


DONE with batch ending with 2022-04-18through2022-04-25


## Get weekly patterns

In [ ]:
transform_dataframe(batch_list[0])

directory: 100%|██████████| 25/25 [25:28<00:00, 61.13s/it]


DONE with batch ending with 2018-06-25through2018-07-02


,node.placekey,node.safegraph_core.naics_code,node.safegraph_core.location_name,node.safegraph_core.top_category,node.safegraph_core.sub_category,node.safegraph_geometry.wkt_area_sq_meters,day_0_visits,day_1_visits,day_2_visits,day_3_visits,...,median_dwell,<5,5-10,11-20,21-60,61-120,121-240,>240,start,end
0,226-225@627-s4v-hkf,722515.0,Dunkin',Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,2110.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-01,2018-01-08
1,zzy-222@627-wgj-qj9,485113.0,MTA New York City Transit BATCHELDER ST VOORHI...,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-01,2018-01-08
2,22r-222@627-wc7-tsq,813110.0,Church of All Nations,Religious Organizations,Religious Organizations,10819.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-01,2018-01-08
3,zzw-22h@627-wby-zcq,5412.0,Arlia & Associates,"Accounting, Tax Preparation, Bookkeeping, and ...",None,11473.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-01,2018-01-08
4,zzy-226@627-s7z-92k,485113.0,MTA New York City Transit PITKIN AV LOGAN ST,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-01,2018-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,22m-222@627-s6g-249,531110.0,Apartments at 306 W 142nd St,Lessors of Real Estate,Lessors of Residential Buildings and Dwellings,113.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-06-25,2018-07-02
496,zzw-223@627-s8r-j7q,522110.0,TD Bank,Depository Credit Intermediation,Commercial Banking,1562.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-06-25,2018-07-02
497,zzy-226@627-vvc-6tv,485113.0,MTA New York City Transit HYLAN BL EBBITTS ST,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-06-25,2018-07-02
498,22f-222@627-s8h-w6k,448210.0,Frye,Shoe Stores,Shoe Stores,33.0,0.0,0.0,0.0,0.0,...,18.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2018-06-25,2018-07-02


In [ ]:
df = transform_dataframe(batch_list[1])

directory: 100%|██████████| 25/25 [25:28<00:00, 61.15s/it]


DONE with batch ending with 2018-12-17through2018-12-24


In [ ]:
df = transform_dataframe(batch_list[2])

directory: 100%|██████████| 25/25 [27:45<00:00, 66.63s/it]


DONE with batch ending with 2019-06-10through2019-06-17


In [ ]:
df = transform_dataframe(batch_list[3])

directory: 100%|██████████| 25/25 [27:34<00:00, 66.16s/it]


DONE with batch ending with 2019-12-02through2019-12-09


In [ ]:
df = transform_dataframe(batch_list[4])

directory: 100%|██████████| 25/25 [26:34<00:00, 63.78s/it]


DONE with batch ending with 2020-05-25through2020-06-01


In [ ]:
df = transform_dataframe(batch_list[5])

directory: 100%|██████████| 25/25 [25:47<00:00, 61.91s/it]


DONE with batch ending with 2020-11-16through2020-11-23


In [ ]:
df = transform_dataframe(batch_list[6])

directory: 100%|██████████| 25/25 [28:44<00:00, 68.98s/it] 


DONE with batch ending with 2021-05-10through2021-05-17


In [ ]:
transform_dataframe(batch_list[7])

directory:  16%|█▌        | 4/25 [02:34<13:55, 39.78s/it]

New York,NYfrom2021-06-14to2021-06-21PART51.json


directory:  32%|███▏      | 8/25 [05:56<14:06, 49.79s/it]

New York,NYfrom2021-07-05to2021-07-12PART138.json
New York,NYfrom2021-07-12to2021-07-19PART7.json


directory:  96%|█████████▌| 24/25 [28:46<01:44, 104.28s/it]

New York,NYfrom2021-11-01to2021-11-08PART49.json


directory: 100%|██████████| 25/25 [30:43<00:00, 73.73s/it] 


DONE with batch ending with 2021-11-01through2021-11-08


,node.placekey,node.safegraph_core.naics_code,node.safegraph_core.location_name,node.safegraph_core.top_category,node.safegraph_core.sub_category,node.safegraph_geometry.wkt_area_sq_meters,day_0_visits,day_1_visits,day_2_visits,day_3_visits,...,median_dwell,<5,5-10,11-20,21-60,61-120,121-240,>240,start,end
0,226-225@627-s4v-hkf,722515.0,Dunkin',Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,2110.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-17,2021-05-24
1,zzy-222@627-wgj-qj9,485113.0,MTA New York City Transit BATCHELDER ST VOORHI...,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-17,2021-05-24
2,22r-222@627-wc7-tsq,813110.0,Church of All Nations,Religious Organizations,Religious Organizations,10819.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-17,2021-05-24
3,zzw-22h@627-wby-zcq,5412.0,Arlia & Associates,"Accounting, Tax Preparation, Bookkeeping, and ...",None,11473.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-17,2021-05-24
4,zzy-226@627-s7z-92k,485113.0,MTA New York City Transit PITKIN AV LOGAN ST,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-17,2021-05-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,zzw-225@627-s4t-5pv,722511.0,137 Bar & Grill,Restaurants and Other Eating Places,Full-Service Restaurants,10192.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-01,2021-11-08
496,229-225@627-wc7-k2k,446191.0,USA Vitamins Com,Health and Personal Care Stores,Food (Health) Supplement Stores,5940.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-01,2021-11-08
497,zzy-227@627-s6h-5s5,485113.0,MTA New York City Transit E BURNSIDE AV GRAND ...,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-01,2021-11-08
498,zzy-222@627-wdf-y5f,485113.0,MTA New York City Transit SURF AV W 35 ST,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-01,2021-11-08


In [ ]:
transform_dataframe(batch_list[8])

directory:   4%|▍         | 1/24 [00:37<14:13, 37.12s/it]

New York,NYfrom2021-11-08to2021-11-15PART70.json


directory: 100%|██████████| 24/24 [32:31<00:00, 81.31s/it] 


DONE with batch ending with 2022-04-18through2022-04-25


,node.placekey,node.safegraph_core.naics_code,node.safegraph_core.location_name,node.safegraph_core.top_category,node.safegraph_core.sub_category,node.safegraph_geometry.wkt_area_sq_meters,day_0_visits,day_1_visits,day_2_visits,day_3_visits,...,median_dwell,<5,5-10,11-20,21-60,61-120,121-240,>240,start,end
0,zzy-22d@627-s4r-wrk,517919.0,LinkNYC,Other Telecommunications,All Other Telecommunications,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-08,2021-11-15
1,225-227@627-wbz-w8v,5616.0,Telestar Locksmiths Inc,Investigation and Security Services,None,146.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-08,2021-11-15
2,22v-222@627-wbz-r6k,445310.0,Simply Wine,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",508.0,1.0,2.0,0.0,2.0,...,43.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2021-11-08,2021-11-15
3,232-223@627-wbv-fzz,722511.0,Magnolia Restaurant,Restaurants and Other Eating Places,Full-Service Restaurants,1268.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-08,2021-11-15
4,zzw-225@627-wc7-4jv,722511.0,Altesi Downtown,Restaurants and Other Eating Places,Full-Service Restaurants,128.0,6.0,3.0,5.0,3.0,...,42.0,0.0,4.0,2.0,15.0,6.0,2.0,3.0,2021-11-08,2021-11-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,232-228@627-s4n-v2k,722511.0,Botte UES,Restaurants and Other Eating Places,Full-Service Restaurants,1972.0,2.0,2.0,1.0,2.0,...,20.0,2.0,4.0,4.0,5.0,0.0,1.0,1.0,2022-04-18,2022-04-25
496,zzy-223@627-wc7-2tv,722410.0,Getaway,Drinking Places (Alcoholic Beverages),Drinking Places (Alcoholic Beverages),407.0,4.0,3.0,4.0,6.0,...,19.5,0.0,8.0,4.0,4.0,3.0,2.0,1.0,2022-04-18,2022-04-25
497,zzw-229@627-wbv-j7q,722511.0,Tavolino,Restaurants and Other Eating Places,Full-Service Restaurants,4670.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-18,2022-04-25
498,239-223@627-wc7-3nq,4238.0,Kiosk,"Machinery, Equipment, and Supplies Merchant Wh...",None,829.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-18,2022-04-25
